In [1]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries
import folium
from shapely.geometry import Point, LineString, shape, mapping
import shapely.wkt
import pyproj
import os
import osmnx as ox
import ogr
import urllib.request
import geopy
from geopy.geocoders import Nominatim
import fiona
import shapefile
from pyproj import Proj, transform

In [2]:
if not os.path.exists('data'): os.makedirs('data')
if not os.path.exists('output'): os.makedirs('output')

Defining functions for getting the coordinates and city shapefile (if wanted).

In [3]:
# Gets the city boundary and saves it as a shapefile, if needed
def city_boundary(city):
    cityboundary = ox.gdf_from_place(city)  # Creates gdf from the place
    ox.save_gdf_shapefile(cityboundary, "city_boundary", "./data")  # Saves gdf as a shapefile

# Gets the coordinates from the city
def get_coordinates(name, city):
    geolocator = Nominatim(user_agent=name)
    location = geolocator.geocode(city)
    lat, lon = location.latitude, location.longitude
    print((lat, lon))
    SeeLocation = folium.Map(location=[lat, lon])
    folium.Marker(location=[lat, lon]).add_to(SeeLocation)
    return lat, lon, SeeLocation

Calling the coordinates function and define the city or place you want to get the lat-lon lines

In [4]:
city = 'Lleida'
name = 'Gerard' # Write your name?
#city_boundary(city) # Call this if you want a shapefile of the city/municipality
lat, lon, AreaMap = get_coordinates(name, city)
#AreaMap # Call this if you want to check if the city/area is correct with a leaflet map

(41.6147605, 0.6267842)


Creating lines with equal latitude and longitude as the selected city

In [5]:
limit_lon = -179.9
limit_lon2 = 179.9

coordinates = {'LAT': [lat, lat], 'LON': [limit_lon, limit_lon2]}

same_lat = pd.DataFrame(data=coordinates)
same_lat = gpd.GeoDataFrame(same_lat, geometry=gpd.points_from_xy(same_lat.LON, same_lat.LAT))
same_lat.crs = {'init': 'epsg:4326'} #epsg4326

#gjson = same_lat.to_json()
geom0_lat = same_lat.loc[0]['geometry']
geom1_lat = same_lat.loc[1]['geometry']

# Create LineString from coordinates
startlat, endlat = [(geom0_lat.x, geom0_lat.y), (geom1_lat.x, geom1_lat.y)]
line_lat = LineString([startlat, endlat])

In [6]:
limit_lat = -89.9
limit_lat2 = 89.9

coordinates = {'LAT': [limit_lat, limit_lat2], 'LON': [lon, lon]}

same_lon = pd.DataFrame(data=coordinates)
same_lon = gpd.GeoDataFrame(same_lon, geometry=gpd.points_from_xy(same_lon.LON, same_lon.LAT))
same_lon.crs = {'init': 'epsg:4326'}

#gjson = same_lat.to_json()
geom0_lon = same_lon.loc[0]['geometry']
geom1_lon = same_lon.loc[1]['geometry']

# Create LineString from coordinates
startlon, endlon = [(geom0_lon.x, geom0_lon.y), (geom1_lon.x, geom1_lon.y)]
line_lon = LineString([startlon, endlon])
print(line_lon)

LINESTRING (0.6267842 -89.90000000000001, 0.6267842 89.90000000000001)


In [7]:
line_lon, line_lat
gs_linelon = gpd.GeoSeries([line_lon])
gs_linelat = gpd.GeoSeries([line_lat])

gslonbuffer = gs_linelon.buffer(0.03)
gslonbuffer.crs = {'init': 'epsg:4326'}

gslatbuffer = gs_linelat.buffer(0.03)
gslatbuffer.crs = {'init': 'epsg:4326'}

Plotting the lines in a leaflet map

In [8]:
mapa = folium.Map(location=[lat, lon])
#points = folium.features.GeoJson(gjson)
#mapa.add_children(points)
#folium.GeoJson(line_lat).add_to(mapa)
#folium.GeoJson(line_lon).add_to(mapa)
folium.GeoJson(gslonbuffer).add_to(mapa)
folium.GeoJson(gslatbuffer).add_to(mapa)

mapa

C:\Users\Gerard\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [9]:
mapa.save("LongLatLleida.html")